attempt at finetuning gpt2 on pushkin poems.
doesn't work yet.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, GPT2LMHeadModel, GPT2Tokenizer

import torch

# display all outputs from Jupyter notebook cells, not just last.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# fetch pretrained model as base to finetune
base_checkpoint = 'ai-forever/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(base_checkpoint)

In [ ]:
# load dataset
dataset = load_dataset("abobster/pushkin_new")

In [ ]:
def tokenize(examples):
    # return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=2048, return_tensors="pt").to(device)
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128, return_tensors="pt").to(device)

tokenized_ds = dataset.map(tokenize, batched=True)

In [ ]:
# finetune parameters

current_model = './fine_tuned_pushkin_gpt2'
previous_model = 'ai-forever/rugpt3small_based_on_gpt2'
output_dir='./results'
num_train_epochs = 2
per_device_batch_size = 4

In [ ]:
model = AutoModelForCausalLM.from_pretrained(previous_model).to(device)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

tr_args = TrainingArguments(
    output_dir=output_dir, 
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size,
    warmup_steps=50,
    weight_decay=0.01,
    save_steps=10_000, 
    save_total_limit=2,
    use_cpu = False,
    resume_from_checkpoint=True
)

trainer = Trainer(
    model=model,
    args=tr_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    data_collator=data_collator)

trainer.train()
trainer.save_model(current_model)

In [ ]:
trainer.evaluate()

In [ ]:
# check results
tokenizer = AutoTokenizer.from_pretrained('ai-forever/rugpt3small_based_on_gpt2')
model = AutoModelForCausalLM.from_pretrained(current_model)

def generate_text(prompt, model, tokenizer, max_length=100, num_return_sequences=1):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        num_beams=5,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        early_stopping=True
    )
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

# Example usage
prompt = "У лукоморья дуб"
generated_texts = generate_text(prompt, model, tokenizer)
for i, text in enumerate(generated_texts):
    print(f'Generated Text {i + 1}:')
    print(text)
    print()